# MinHash

## In this question you will implement a MinHash class to generate a signature to calculate jaccard similarity between sets.
### [Part 1] Please follow the following steps
1. Fill in generate_signature function using the hash_functions list
2. Implement jaccard_similarity function that takes two minhash signatures as input and returns their estimated similarity
3. Implement jaccard_original function, which calculates the exact jaccard similarity by using the original elements of a list

In [49]:
import random
import pandas as pd
random.seed(0)
class MinHash:
    def __init__(self, num_hashes):
        self.num_hashes = num_hashes
        self.hash_functions = self._generate_hash_functions()

    def _generate_hash_functions(self):
        def _hash(x):
            return hash(str(x))
        
        hash_functions = []
        for i in range(self.num_hashes):
            next_prime = self._next_prime(i)
            hash_functions.append(lambda x, i=i: (_hash(x) + i) % next_prime)
        return hash_functions

    def _next_prime(self, n):
        def is_prime(num):
            if num < 2:
                return False
            for i in range(2, int(num ** 0.5) + 1):
                if num % i == 0:
                    return False
            return True

        prime = n
        while not is_prime(prime):
            prime += 1
        return prime
    
    def generate_signature(self, lst):
        #This function generates a minhash signature for the elements in lst
        #Approach: 
            #Think of hash function as returning the rank of elements in the permutation
            #Element of lst that has the minimum hash is the first element in the permutation that belongs to lst
        signature = []
        for hash_function in self.hash_functions:
            min_hash = float('inf')
            for element in lst:
                hash_value = hash_function(element)
                if hash_value < min_hash:
                    min_hash = hash_value
            signature.append(min_hash)
        return signature

def jaccard_similarity(minhash1, minhash2):

    assert len(minhash1) == len(minhash2)
    num_agree = 0
    for i in range(len(minhash1)):
        if minhash1[i] == minhash2[i]:
            num_agree += 1
    return num_agree / len(minhash1)


def jaccard_original(lst1,lst2):
    set1 = set(lst1)
    set2 = set(lst2)
    return len(set1.intersection(set2)) / len(set1.union(set2))



# [Part 2] Test Minhash on the following datasets and choose num_hashes

Pick num_hashes such that the estimated jaccard similarity is within 0.10 of the original similarity. You can just vary num_hashes to pick the smallest value which ensures difference less than 0.10



In [50]:
#Please give the path of downloaded files here
path = '/Users/zq84/CS6386/HW1/datarepo/'
files=['5kru-rjmf.csv','23z9-6uk9.csv','29bv-qqsy.csv','5g59-fxev.csv','29ry-u5bf.csv']

low, high = 2600, 2700
while low < high:
    mid = (low + high) // 2
    print("Trying with num_hashes = ", mid)
    minhash = MinHash(mid)
    hash_lst = []
    val_lst = []
    for file in files:
        df = pd.read_csv(path + file)
        if 'DBN' in df.columns:
            hashval = minhash.generate_signature(list(df['DBN'].values))
            val_lst.append(list(df['DBN'].values))
        else:
            hashval = minhash.generate_signature(list(df['dbn'].values))
            val_lst.append(list(df['dbn'].values))
        hash_lst.append(hashval)

    all_within_tolerance = True
    for i in range(len(hash_lst)):
        for j in range(i + 1, len(hash_lst)):
            min_hash_jaccard = jaccard_similarity(hash_lst[i], hash_lst[j])
            original_jaccard = jaccard_original(val_lst[i], val_lst[j])
            if abs(min_hash_jaccard - original_jaccard) > 0.1:
                all_within_tolerance = False
                break
        if not all_within_tolerance:
            break

    if all_within_tolerance:
        high = mid
    else:
        low = mid + 1

num_hashes = low
minhash = MinHash(num_hashes)


#Calculates jaccard similarity
hash_lst=[]
val_lst=[]
for file in files:
    df=pd.read_csv(path+file)
    if 'DBN' in df.columns:
        hashval=minhash.generate_signature(list(df['DBN'].values))
        val_lst.append(list(df['DBN'].values))
    else:
        hashval=minhash.generate_signature(list(df['dbn'].values))
        val_lst.append(list(df['dbn'].values))
    hash_lst.append(hashval)

print ("Below are the estimated and original similarities")
print ("i, j, estimated sim, accurate sim")
i=0
while i<len(hash_lst):
    j=i+1
    while j<len(hash_lst):
        min_hash_jaccard=jaccard_similarity(hash_lst[i], hash_lst[j])
        original_jaccard=jaccard_original(val_lst[i], val_lst[j])
        if abs(min_hash_jaccard-original_jaccard)>0.1:
            print (i,j,min_hash_jaccard,original_jaccard, "Difference is more than 0.1")
        else:
            print (i,j,min_hash_jaccard,original_jaccard, "Difference is less than 0.1")
        j+=1
    i+=1



Trying with num_hashes =  2650
Trying with num_hashes =  2675
Trying with num_hashes =  2688
Trying with num_hashes =  2694
Trying with num_hashes =  2697
Trying with num_hashes =  2696
Trying with num_hashes =  2695
Below are the estimated and original similarities
i, j, estimated sim, accurate sim
0 1 0.3068645640074211 0.25586995785671285 Difference is less than 0.1
0 2 0.9335807050092765 0.9242081447963801 Difference is less than 0.1
0 3 0.022634508348794064 0.010676156583629894 Difference is less than 0.1
0 4 0.7272727272727273 0.6806470940683044 Difference is less than 0.1
1 2 0.288682745825603 0.24426605504587157 Difference is less than 0.1
1 3 0.033024118738404454 0.0 Difference is less than 0.1
1 4 0.15064935064935064 0.05783456624075319 Difference is less than 0.1
2 3 0.013358070500927645 0.0 Difference is less than 0.1
2 4 0.700556586270872 0.6565366972477065 Difference is less than 0.1
3 4 0.012244897959183673 0.0 Difference is less than 0.1


# [Part 3] We will now use Minhash to implement LSH
Given: The function has already initialized 1 bucket for each band (in __init__ function)
1. Implement index function to add each dataset to a bucket for each band
    You need to do the following
    a. Split signature into different bands
    b. Map each band to a bucket
2. Implement the query function that identifies all buckets for a signature and returns LSHcandidates.
3. Test with '5kru-rjmf.csv' as the query to identify datasets that have more than 0.90 similarity with the DBN column of '5kru-rjmf.csv'. 
4. Show the advantage of LSH over the naive quadratic algorithm to calculate most similar pairs of datasets.

In [51]:
import random
import itertools

class LSH:
    def __init__(self, num_bands, band_size):
        self.num_bands = num_bands
        self.band_size = band_size
        self.buckets = [{} for _ in range(num_bands)]

    def _split_signature(self, minhash_signature):
        return [
                minhash_signature[i:i+self.band_size]
                for i in range(0, len(minhash_signature), self.band_size)
               ]

    #This function takes a dataset_id and its minhash signature
    #The function adds these datasets to respective buckets
    def index(self, dataset_id, minhash_signature):
        band_keys = self._split_signature(minhash_signature)
        for i, band_key in enumerate(band_keys):
            band_key = tuple(band_key)
            if band_key not in self.buckets[i]:
                self.buckets[i][band_key] = []
            self.buckets[i][band_key].append(dataset_id)

        
    #This function takes minhash signature as input and returns a list of candidates
    #that share the same bucket as this dataset
    def query(self, minhash_signature):
        band_keys = self._split_signature(minhash_signature)
        candidates = set()
        for i, band_key in enumerate(band_keys):
            band_key = tuple(band_key)
            if band_key in self.buckets[i]:
                for candidate in self.buckets[i][band_key]:
                    candidates.add(candidate)
        return list(candidates)



In [55]:
#You don't have to change num_hashes and num_bands
#However, if your results do not show advantage of LSH over the naive algorithm, 
# you can change these to suit your needs
num_hashes = 1000
num_bands = 10
minhash = MinHash(num_hashes)
band_size = num_hashes // num_bands
lsh = LSH(num_bands, band_size)
files=['5kru-rjmf.csv','23z9-6uk9.csv','29bv-qqsy.csv','5g59-fxev.csv','29ry-u5bf.csv']


In [56]:
query_file = '5kru-rjmf.csv'
query_df = pd.read_csv(path + query_file)
if 'DBN' in query_df.columns:
    query_dbn = list(query_df['DBN'].values)
else:
    query_dbn = list(query_df['dbn'].values)

query_signature = minhash.generate_signature(query_dbn)

for idx, file in enumerate(files):
    if file == query_file:
        continue
    df = pd.read_csv(path + file)
    if 'DBN' in df.columns:
        dbn_list = list(df['DBN'].values)
    else:
        dbn_list = list(df['dbn'].values)
    signature = minhash.generate_signature(dbn_list)
    lsh.index(idx, signature)

candidates = lsh.query(query_signature)

print("LSH Candidates with more than 0.90 similarity:")
for candidate in candidates:
    candidate_dbn = val_lst[candidate]
    similarity = jaccard_original(query_dbn, candidate_dbn)
    if similarity > 0.90:
        print(f"File: {files[candidate]}, Similarity: {similarity}")

LSH Candidates with more than 0.90 similarity:
File: 29bv-qqsy.csv, Similarity: 0.9242081447963801


In [58]:
import time

all_dbn_lists = []

for idx, file in enumerate(files):
    df = pd.read_csv(path + file)
    if 'DBN' in df.columns:
        dbn_list = list(df['DBN'].values)
    else:
        dbn_list = list(df['dbn'].values)
    all_dbn_lists.append(dbn_list)

all_signature = []
for key, dbn_list in enumerate(all_dbn_lists):
    signature = minhash.generate_signature(dbn_list)
    all_signature.append(signature)
    lsh.index(key, signature)

# Measure time taken by LSH
start_time = time.time()

most_similar_score = 0
most_similar_pair = None

for key, dbn_list in enumerate(all_dbn_lists):
    signature = all_signature[key]
    candidates = lsh.query(signature)
    for candidate in candidates:
        if candidate == key:
            continue
        candidate_dbn_list = all_dbn_lists[candidate]
        similarity = jaccard_original(dbn_list, candidate_dbn_list)
        if similarity > most_similar_score:
            most_similar_score = similarity
            most_similar_pair = (files[key], files[candidate])
lsh_time = time.time() - start_time

print("Most similar pair of files using LSH:", most_similar_pair, "with similarity", most_similar_score)
print("Time taken by LSH:", lsh_time, "seconds")

# Measure time taken by naive quadratic algorithm
start_time = time.time()
most_similar_score = 0
most_similar_pair = None
for i in range(len(files)):
    for j in range(i + 1, len(files)):
        similarity = jaccard_original(val_lst[i], val_lst[j])
        if similarity > most_similar_score:
            most_similar_score = similarity
            most_similar_pair = (files[i], files[j])
naive_time = time.time() - start_time

print("Most similar pair of files using naive quadratic algorithm:", most_similar_pair, "with similarity", most_similar_score)
print("Time taken by naive quadratic algorithm:", naive_time, "seconds")

Most similar pair of files using LSH: ('5kru-rjmf.csv', '29bv-qqsy.csv') with similarity 0.9242081447963801
Time taken by LSH: 0.0007302761077880859 seconds
Most similar pair of files using naive quadratic algorithm: ('5kru-rjmf.csv', '29bv-qqsy.csv') with similarity 0.9242081447963801
Time taken by naive quadratic algorithm: 0.0016188621520996094 seconds
